# ✔Pytorch Tutorial

## 1. QuickStart


---


<br>

### - 데이터 작업하기

<br>

**파이토치의 데이터 작업을 위한 두가지 요소**

- torch.utils.data.Dataset
> Dataset: Sample과 정답(label)을 저장하고 있는 데이터 덩어리(?)

- torch.utils.data.DataLoader
> DataLoader: Dataset을 돌아다닐수 있게 순회 가능한 객체(iterable)로 감싸는 거



In [ ]:
#추가 공부 참고 사이트: https://webnautes.tistory.com/2074

#파이토치는 TorchText등과 같이 도메인 특화 라이브러리(특정 도메인(영역, 작업)에 적합한 언어. 예 - SQL)를 제공함.
import torch
from torch import nn #신경망 구축에 필요한 다양한 요소들 제공
from torch.utils.data import DataLoader
from torchvision import datasets #torchvision.dataset: CIFAR, COCO등 Vision data에 대한 dataset을 포함함
# vision data: 영상, 이미지 관련 데이터
from torchvision.transforms import ToTensor

### **- *FasionMNIST* 사용해서 튜토리얼 해보기**

모든 TorchVision Dataset은 샘플과 정답을 변경하기 위한 transform과 target_transform 인자가 있음.

In [ ]:
# 공개 데이터셋에서 학습 데이터를 내려받습니다.
training_data = datasets.FashionMNIST(
    root="./", #root 디렉토리
    train = True, #If True, creates dataset from ``train-images-idx3-ubyte``
    download=True, # If True, downloads the dataset from the internet and puts it in root directory.
    transform = ToTensor()
)

# 공개 데이터셋에서 테스트 데이터를 내려받습니다.
test_data = datasets.FashionMNIST(
    root="./",
    train=False,
    download=True,
    transform=ToTensor(), # A function/transform that  takes in an PIL image and returns a transformed version.
)



100%|██████████| 26421880/26421880 [00:01<00:00, 17676848.72it/s]


Extracting ./FashionMNIST/raw/train-images-idx3-ubyte.gz to ./FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 318928.28it/s]


Extracting ./FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 5540957.40it/s]


Extracting ./FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 11608751.07it/s]


Extracting ./FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./FashionMNIST/raw



**- Dataset을 DataLoader에 인자로 주기.**

> Dataset을 순회 가능 객체로 감싸기, 배치(batch) 자동화, 샘플링, 섞기(suffle)및 다중 프로세스로 데이터 불러오기(multiprocess data loading, 병렬로 처리, batch로 일괄적으로 모아서 한꺼번에 처리)지원.
</br>

이 예제에서는 batch size를 64로 정의함.

즉, dataloader객체의 각 요소들이 64개의 feature와 label을 하나의 batch로 반환함.

In [ ]:
batch_size = 64

#데이터로더를 생성합니다.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}") # x텐서의 모양. #64개, channel 하나, 28x28 이미지
    print(f"Shape of y:{y.shape} {y.dtype}") # y tensor의 data type

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y:torch.Size([64]) torch.int64
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y:torch.Size([64]) torch.int64
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y:torch.Size([64]) torch.int64
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y:torch.Size([64]) torch.int64
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y:torch.Size([64]) torch.int64
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y:torch.Size([64]) torch.int64
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y:torch.Size([64]) torch.int64
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y:torch.Size([64]) torch.int64
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y:torch.Size([64]) torch.int64
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y:torch.Size([64]) torch.int64
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y:torch.

Pytorch에서 신경망 모델은 nn.Module을 상속받는 class를 생성해서 정의함.

* __init__함수: 신경망 계층(layer)정의
* forward 함수: 신경망에 데이터를 어떻게 전달할지 지정. 가능한 경우 GPU나 MPS로 신경망을 이동시켜 연산올 가속(accelerate)함.

In [ ]:
# 학습에 사용할 CPU나 GPU, MPS 장치를 얻습니다.
device = (
    "cuda" # GPU에서 병렬처리 알고리즘을 C등 산업 표준 언어를 사용해 작성할 수 있도록 하는 GPGPU기술
    if torch.cuda.is_available() #cuda 못 쓰면
    else "mps" #mps 쓸게
    #mps: Multi-Process Service
    if torch.backends.mps.is_available() #그것도 못 쓰면
    else "cpu" #cpu 쓸게
)

print(f"Using {device} device")

#모델을 정의합니다.
class NeuralNetwork(nn.Module): # nn.Modeule 상속
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten() #다차원 배열 공간을 1차원으로 평탄화해주는 함수
        self.linear_relu_stack = nn.Sequential( #nn.Sequential(): nn.Linear, nn.ReLU(활성화 함수) 같은 모듈들을 인수로 받아서 순서대로 정렬해놓고 입력값이 들어모면 순서대로 모듈을 실행해서 결과값을 리턴
            nn.Linear(28*28, 512), # y = Wx + b 선형 변환을 위한 입력 개수(28*28)과 출력 개수(512)를 설정.
            nn.ReLU(), # 활성화 함수로 비선형 함수인 ReLU를 사용
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x): # 앞에서 미리 선언해놓은 레이어들을 사용하여 입력에서 출력까지 모델을 생성
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device) #.to == ANY
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


### **- 모델 매개변수 최적화하기**

모델 학습을 위해서는 **손실 함수(loss fuction)**와 **옵티마이저(optimiser)**가 필요함.

> 딥러닝 모델은 손실 함수의 최소값을 찾는 것을 목표로 하는데,

> 최소값을 찾아가는 것 = optimization 이고,

> optimization을 위한 알고리즘 = optimizer임.

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

각 학습 단계(training loop)마다 **batch단위**로 오는 학습 데이터셋을 모델을 통과시켜 결과를 예측하고,

정확한 예측을 위해, **예측된 결과값의 오류를 역전파**시켜 모델의 매개변수를 조정함.

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch,(X, y) in enumerate(dataloader): #batch는 몇 번 했는지 보려고
        X, y = X.to(device), y.to(device)

        # 예측 오류 계산
        pred = model(X) #결과 예측
        loss = loss_fn(pred, y) #손실 함수 정의

        # 역전파
        optimizer.zero_grad()  # 모든 Tensor의 그래디언트를 0으로 초기화함.
                               # 해주지 않으면 그레디언트가 누적되는 문제가 생김.
        loss.backward() #역전파
        optimizer.step() #한 step씩 진행

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

# 테스트 데이터셋으로 모델 성능 확인하는 함수 정의
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset) #전체 데이터 셋 사이즈
    num_batches = len(dataloader) #batch 갯수 = dataloader 길이
    model.eval() #PyTorch에서 모델을 평가 모드로 전환하는 메서드.
                 #드롭아웃이 비활성화되고 배치 정규화의 이동 평균과 이동 분산이 업데이트되지 않음.
                 #테스트 데이터나 검증 데이터를 사용하여 모델을 평가할 때 사용
                 #모델 객체를 추론(inference)할 때 model.eval()을 호출하여 추론 모드로 전환하고,
                 #추론이 끝난 후에는 다시 model.train()을 호출하여 학습 모드로 전환하는 것이 일반적.
    test_loss, correct = 0, 0
    with torch.no_grad(): #자원을 획득하고, 사용하고, 반납할때 주로 사용
        for X, y in dataloader:
            X, y = X.to(device), y.to(device) #torch.tensor.to(): tensor를 지정한 device로 보냄.
            pred = model(X)
            test_loss += loss_fn(pred, y).item() #.item(): 딕셔너리에 사용, key, value쌍을 가져올 때 사용
            correct += (pred.argmax(1) == y).type(torch.float).sum().item() #pred.argmax(1) == nn.argmax(pred, dim=1), 행을 기준으로 max 값이 있는 index를 출력
                                                                            #예측 값이 y(label)값이랑 같으면
                                                                            #그걸 float으로 형변환 해서
                                                                            #더하고
                                                                            #key,value 쌍 값을 가져온다
        test_loss /= num_batches #batch당 발생한 오류의 평균
        correct /= size #총 결과 예측이 맞은 확률(정답 맞춘 개수 / 전체 샘플 개수)
        print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss:{test_loss:>8f} \n")



학습 단계는 여러번의 반복 단계(epochs)를 거쳐 수행됨.

각 epochs에서 더 나은 예측을 위해 매개변수가 학습됨.

In [ ]:
#각 epochs마다 정확도(accuracy)와 손실(loss) 출력.
#epoch마다 정확도가 증가하고 손실이 감소하는지 확인하기 위함

epochs = 5 #학습을 5번 돌린다

for t in range(epochs):
    print(f"Epoch {t+1}\n----------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")


Epoch 1
----------------------------
loss: 1.142205 [   64/60000]
loss: 1.161456 [ 6464/60000]
loss: 0.967523 [12864/60000]
loss: 1.117135 [19264/60000]
loss: 0.986845 [25664/60000]
loss: 1.006793 [32064/60000]
loss: 1.052057 [38464/60000]
loss: 0.994590 [44864/60000]
loss: 1.032449 [51264/60000]
loss: 0.966538 [57664/60000]
Test Error: 
 Accuracy: 66.5%, Avg loss:0.976673 

Epoch 2
----------------------------
loss: 1.019979 [   64/60000]
loss: 1.061606 [ 6464/60000]
loss: 0.850415 [12864/60000]
loss: 1.021950 [19264/60000]
loss: 0.895408 [25664/60000]
loss: 0.912465 [32064/60000]
loss: 0.975252 [38464/60000]
loss: 0.920680 [44864/60000]
loss: 0.955130 [51264/60000]
loss: 0.899433 [57664/60000]
Test Error: 
 Accuracy: 67.8%, Avg loss:0.903631 

Epoch 3
----------------------------
loss: 0.931217 [   64/60000]
loss: 0.993123 [ 6464/60000]
loss: 0.767842 [12864/60000]
loss: 0.956460 [19264/60000]
loss: 0.835692 [25664/60000]
loss: 0.845064 [32064/60000]
loss: 0.922529 [38464/60000]
loss

### **- 모델 저장하기**

일반적인 방법은 모델의 매개변수들을 포함해서 **내부 상태 사전(internal state dictionary)를 직렬화(serialize)**하는 것.

In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved Pytorch Model State to model.pth")

Saved Pytorch Model State to model.pth


모델 불러오기
모델을 불러오는 과정에는 **모델 구조를 다시 만들고**, **상태 사전을 모델에 불러오는 과정**이 포함됨.

In [ ]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [ ]:
# 불러온 모델로 결과 예측하기

classes =[
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot"
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad(): # 주된 목적은 autograd를 끔으로써 메모리 사용량을 줄이고 연산 속도를 높히기 위함
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]

    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
